<a href="https://colab.research.google.com/github/nonameretard/DummyCNNforBoris/blob/master/DummyCNNforBoris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# To determine which version you're using:
!pip show tensorflow

# For a specific version:
# !pip install tensorflow

In [50]:
# 2018.10.29
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
# 2018.11.20
# https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
# 2018.11.23
# https://github.com/jacobkimmel/lanternfish/blob/master/bestiary.py
# 2018.11.29
# https://github.com/keras-team/keras/blob/master/keras/datasets/cifar10.py

import tensorflow as tf
import keras
import numpy as np
import skimage.color

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data ()

# Normalize data
x_train = x_train.astype ('float32') / 255
x_test = x_test.astype ('float32') / 255

# test_pixel
test_pixel = x_test [1:3]
test_pixel [1] [1] [1] = (1, 1, 1)
test_pixel [1] [1] [2] = (1, 1, 0)
test_pixel [1] [1] [3] = (1, 0, 1)
test_pixel [1] [1] [4] = (0, 1, 1)
test_pixel [1] [1] [5] = (1, 0, 0)
test_pixel [1] [1] [6] = (0, 1, 0)
test_pixel [1] [1] [7] = (0, 0, 1)
test_pixel [1] [1] [8] = (0, 0, 0)

def convert (x):
    for i in range (x.shape [0]):
        x [i] = (skimage.color.rgb2ypbpr (x [i]) + (
            0, 0.5, 0.5) ) / (
            1, 1, 1) * 2 - 1
    return x

print ('test_pixel')
test_pixel = convert (test_pixel)
print (test_pixel [1] [1] [1:9])
print ('convert x_train')
x_train = convert (x_train)
print ('convert x_test')
x_test = convert (x_test)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical (y_train, num_classes)
y_test = keras.utils.to_categorical (y_test, num_classes)

test_pixel
[[ 1.        0.        0.      ]
 [ 0.772    -1.        0.162624]
 [-0.174     0.662528  0.837376]
 [ 0.402     0.337472 -1.      ]
 [-0.402    -0.337472  1.      ]
 [ 0.174    -0.662528 -0.837376]
 [-0.772     1.       -0.162624]
 [-1.        0.        0.      ]]
convert x_train
convert x_test


In [45]:
batch_size = 512
epochs = 10

def frac_max_pool (x):
    import tensorflow as tf
    return tf.nn.fractional_max_pool (
        x, pooling_ratio = [1., 1.44, 1.73, 1.]) [0]

# Define functional model
inputLayer = keras.layers.Input (
    shape = (x_train.shape [1], x_train.shape [2], x_train.shape [3]))
x = keras.layers.Conv2D (16, 4) (inputLayer)
x = keras.layers.Conv2D (16, 4, activation = 'relu') (x)
x = keras.layers.Lambda (frac_max_pool) (x)
x = keras.layers.Flatten () (x)
x = keras.layers.Dense (512, activation = 'relu') (x)
x = keras.layers.GaussianDropout (1./4) (x)
x = keras.layers.Dense (num_classes, activation = 'softmax') (x)

model = keras.models.Model (inputs = inputLayer, outputs = x)

model.compile (
    optimizer = keras.optimizers.Nadam (beta_2 = 2./3, schedule_decay = 1./3),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
model.fit (
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 1,
    validation_data = (x_test, y_test)
)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 10s 209us/step - loss: 1.5661 - acc: 0.4447 - val_loss: 1.2472 - val_acc: 0.5619
Epoch 2/10
50000/50000 [==============================] - 10s 191us/step - loss: 1.1810 - acc: 0.5876 - val_loss: 1.0909 - val_acc: 0.6175
Epoch 3/10
50000/50000 [==============================] - 10s 191us/step - loss: 1.0291 - acc: 0.6409 - val_loss: 1.0055 - val_acc: 0.6470
Epoch 4/10
50000/50000 [==============================] - 9s 190us/step - loss: 0.9473 - acc: 0.6688 - val_loss: 0.9798 - val_acc: 0.6591
Epoch 5/10
50000/50000 [==============================] - 9s 189us/step - loss: 0.8803 - acc: 0.6919 - val_loss: 0.9469 - val_acc: 0.6713
Epoch 6/10
50000/50000 [==============================] - 9s 189us/step - loss: 0.8233 - acc: 0.7120 - val_loss: 0.9247 - val_acc: 0.6802
Epoch 7/10
50000/50000 [==============================] - 9s 188us/step - loss: 0.7811 - acc: 0.7285 - val_loss: 0.9116

KeyboardInterrupt: ignored

In [0]:
model.save ('my_model.h5')

In [0]:
model1 = keras.models.load_model ('my_model.h5')
score = model1.evaluate (x_test, y_test, verbose = 0)
print ('Test loss:', score [0])
print ('Test accuracy:', score [1])

Test loss: 0.8778221111297607
Test accuracy: 0.7012
